In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE

In [ ]:
# 读取 Excel 文件
df_data = pd.read_excel('ResearchData/NM/NM-ResearchData-Total.xlsx')
display(df_data)

In [ ]:
# 选择特征列
X = df_data.iloc[:, list(range(4, 14)) + list(range(26, 29))]  
display(X)

In [ ]:
# 选择预测列
y = df_data.iloc[:, 38] 
display(y)

In [ ]:
# 1. 使用SMOTE对少数类样本进行过采样
smote = SMOTE(sampling_strategy='auto', random_state=42)  # 自动平衡正负样本
X_resampled, y_resampled = smote.fit_resample(X, y)

In [50]:
# 2. 分割数据为训练集和测试集（8:2）
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [51]:
# 评估函数：用于输出准确率、精确率、召回率和F1分数
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return acc, precision, recall, f1

In [52]:
# 创建一个空字典用于保存模型的评估结果
results = {
    'Model': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1-Score': []
}

In [ ]:
# 3. Logistic Regression
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
logreg_results = evaluate_model(logreg, X_test, y_test)
results['Model'].append('Logistic Regression')
results['Accuracy'].append(logreg_results[0])
results['Precision'].append(logreg_results[1])
results['Recall'].append(logreg_results[2])
results['F1-Score'].append(logreg_results[3])

In [10]:
# 4. Random Forest classifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Evaluate the model
rf_results = evaluate_model(rf, X_test, y_test)

# Store the results
results['Model'].append('Random Forest')
results['Accuracy'].append(rf_results[0])
results['Precision'].append(rf_results[1])
results['Recall'].append(rf_results[2])
results['F1-Score'].append(rf_results[3])

In [11]:
# 5. K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_results = evaluate_model(knn, X_test, y_test)
results['Model'].append('KNN')
results['Accuracy'].append(knn_results[0])
results['Precision'].append(knn_results[1])
results['Recall'].append(knn_results[2])
results['F1-Score'].append(knn_results[3])

In [12]:
# 6. Naive Bayes classifier
nb = GaussianNB()
nb.fit(X_train, y_train)
# Evaluate the model
nb_results = evaluate_model(nb, X_test, y_test)
# Store the results
results['Model'].append('Naive Bayes')
results['Accuracy'].append(nb_results[0])
results['Precision'].append(nb_results[1])
results['Recall'].append(nb_results[2])
results['F1-Score'].append(nb_results[3])

In [ ]:
# 7. XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb.fit(X_train, y_train)
xgb_results = evaluate_model(xgb, X_test, y_test)
results['Model'].append('XGBoost')
results['Accuracy'].append(xgb_results[0])
results['Precision'].append(xgb_results[1])
results['Recall'].append(xgb_results[2])
results['F1-Score'].append(xgb_results[3])

In [ ]:
# 8. LightGBM
lgbm = lgb.LGBMClassifier()
lgbm.fit(X_train, y_train)
lgbm_results = evaluate_model(lgbm, X_test, y_test)
results['Model'].append('LightGBM')
results['Accuracy'].append(lgbm_results[0])
results['Precision'].append(lgbm_results[1])
results['Recall'].append(lgbm_results[2])
results['F1-Score'].append(lgbm_results[3])

In [ ]:
# 将字典转换为DataFrame
results_df = pd.DataFrame(results)
display(results_df)

In [ ]:
# results_df.to_excel("ResultsData/NM-ClassifyModelQuality.xlsx", index=False)

In [ ]:
import shap
import matplotlib.pyplot as plt

In [18]:
# 假设模型已经训练好，X 是你的数据集
explainer = shap.TreeExplainer(xgb)

# 获取 SHAP 值
shap_values_numpy = explainer.shap_values(X_test)

In [ ]:
# 创建图形对象
fig, ax1 = plt.subplots(figsize=(24, 8), dpi=1000)

# 绘制 Dot 图
shap.summary_plot(shap_values_numpy, X_test, feature_names=X_test.columns, plot_type="dot", show=False, color_bar=True)
plt.gca().set_position([0.5, 0.5, 0.65, 0.65])  # 设置位置
ax1 = plt.gca()  # 获取第一个子图

# 创建第二个 x 轴
ax2 = ax1.twiny()

# 绘制 Bar 图
shap.summary_plot(shap_values_numpy, X_test, plot_type="bar", show=False)

# 设置第二个子图的位置
plt.gca().set_position([0.5, 0.5, 0.65, 0.65])


# 获取所有的柱状图对象
bars = ax2.patches  # 获取所有的柱状图对象
for bar in bars:
    bar.set_alpha(0.2)  # 设置透明度

# 设置标签
ax1.set_xlabel('Shapley Value Contribution')
ax2.set_xlabel('Mean Shapley Value')
ax2.xaxis.set_label_position('top')  # 将标签移动到顶部
ax2.xaxis.tick_top()  # 将刻度也移动到顶部

# 显式设置 ax2 的边框线，确保可见
ax2.spines['top'].set_visible(True)  # 显示上边框
ax2.spines['bottom'].set_visible(True)  # 显示下边框
ax2.spines['left'].set_visible(True)  # 显示左边框
ax2.spines['right'].set_visible(True)  # 显示右边框

# 设置坐标轴颜色为透明或其他颜色（例如：white）
ax1.spines['top'].set_color('black')  # 上边框颜色设置为黑色
ax1.spines['bottom'].set_color('black')  # 下边框颜色设置为黑色
ax2.spines['top'].set_color('black')  # 上边框颜色设置为黑色
ax2.spines['bottom'].set_color('black')  # 下边框颜色设置为黑色

# 添加边框线给 ax2
ax2.spines['top'].set_linewidth(1)  # 设置上边框线的宽度
ax2.spines['bottom'].set_linewidth(1)  # 设置下边框线的宽度
ax2.spines['top'].set_color('black')  # 设置颜色为黑色
ax2.spines['bottom'].set_color('black')  # 设置颜色为黑色

# 设置刻度线颜色为黑色
ax1.tick_params(axis='x', colors='black')  # 设置 X 轴刻度线颜色为黑色
ax1.tick_params(axis='y', colors='black')  # 设置 Y 轴刻度线颜色为黑色
ax2.tick_params(axis='x', colors='black')  # 设置第二个 X 轴刻度线颜色为黑色
ax2.tick_params(axis='y', colors='black')  # 设置第二个 Y 轴刻度线颜色为黑色


# 调整布局
plt.tight_layout()

# 保存文件
plt.savefig("Figure/NM-SHAP-XGB.jpg", format='jpg', dpi=1000, bbox_inches='tight')

# 显示图形
plt.show()


In [20]:
from scipy.optimize import curve_fit
from matplotlib.ticker import FormatStrFormatter
import numpy as np

In [21]:
# Define quadratic polynomial function for curve fitting
def polynomial_func(x, a, b, c):
    return a * x ** 2 + b * x + c

In [22]:
# 假设 feature_names 是包含所有特征名称的列表
feature_names = X_test.columns.tolist()  # 或者你有一个自定义的特征名称列表

In [ ]:
# 设置子图数量
num_features = shap_values_numpy.shape[1]

# 创建子图，设置子图网格（假设32个特征，每行8个特征，共4行）
fig, axes = plt.subplots(nrows=8, ncols=4, figsize=(20, 32), dpi=600)

# 展开 axes 数组（以便可以用 1D 索引）
axes_flat = axes.flatten()

# 循环遍历每个特征
for i in range(num_features):
    # 选择第i个特征的 SHAP 值
    x_data = X_test.iloc[:, i]  # 选择第i个特征的数据
    y_data = shap_values_numpy[:, i]  # 获取第i个特征的 SHAP 值
    
    # 获取当前的子图
    current_ax = axes_flat[i]

    # 绘制散点图
    current_ax.scatter(x_data, y_data, color='blue', alpha=0.2, label="SHAP Values", s=10)

    # 执行拟合
    # 对数据进行二次多项式拟合
    popt, pcov = curve_fit(polynomial_func, x_data, y_data)

    # 生成用于绘制拟合曲线的x值
    x_fit = np.linspace(x_data.min(), x_data.max(), 100)

    # 计算拟合曲线对应的y值
    y_fit = polynomial_func(x_fit, *popt)

    # 绘制拟合曲线
    current_ax.plot(x_fit, y_fit, color='red', linewidth=5, label="Fitted Curve", alpha=0.3)

    # 设置标签
    current_ax.set_xlabel(f"{feature_names[i]}", fontsize=22)  # 使用特征名称
    current_ax.set_ylabel('SHAP Value', fontsize=22)
    # current_ax.set_title(f"{feature_names[i]}", fontsize=10)  # 使用特征名称作为标题

    # 移除网格线
    current_ax.grid(False)

    # 设置坐标轴刻度标签的字体大小
    current_ax.tick_params(axis='both', which='major', labelsize=18)

    # 加粗散点图的图框（坐标轴线）
    for spine in current_ax.spines.values():
        spine.set_linewidth(1.5)

# 处理空子图（如果有）
for i in range(num_features, len(axes_flat)):
    axes_flat[i].axis('off')  # 关闭空的子图

# 调整布局以避免重叠
plt.tight_layout()

# 保存图像为 JPG 文件
plt.savefig("Figure/NM-SHAPDependencyAnalysis.jpg", format='jpg', dpi=600, bbox_inches='tight')

# 显示图像
plt.show()